In [1]:
word2info = {}
import glob
filenames = glob.glob('./词典七类/*txt') 
for filename in filenames:
    name = filename.split('\\')[-1].split('.')[0]
    with open(filename, encoding='utf-8') as fr:
        for line in fr:
            word2info.setdefault(name, []).append(line.strip())

In [2]:
ALLKEYS = []
for ls in word2info.values():
    ALLKEYS.extend(ls)

In [3]:
ALLKEYS = set(ls)

In [4]:
word2info.keys()

dict_keys(['乐', '哀', '好', '怒', '恶', '惊', '惧'])

In [5]:
import numpy as np
from collections import defaultdict
def getData(datas):
    label2num = defaultdict(int)
    num = 1 #总情感词数
    for k, v in word2info.items():
        for _ in v:
            if _ in ''.join(datas):
                num+=1
                label2num[k] = label2num.setdefault(k, 0) +1
            else:
                pass
    emos = ['乐','好','怒','衰','惧','恶','惊']
    ifos = label2num['乐'], label2num['好'], label2num['怒'], label2num['衰'], label2num['惧'], label2num['恶'], label2num['惊']
    return datas, label2num['乐'], label2num['好'], label2num['怒'], label2num['衰'], label2num['惧'], label2num['恶'], label2num['惊'], emos[np.argmax(ifos)]

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('文本去重汇总.csv', encoding='utf-8')

In [8]:
df.head()

,内容
0,打工人，再也不用点外卖了！有了它，想吃啥就吃啥提前注水，开机加热半小时就可以吃到麻麻做的热乎...
1,宝宝辅食山药虾泥这个味道真的很好我家不爱吃辅食的宝宝都喜欢?-好吃-宝妈们可以尝试一下哦山药...
2,上火了，口苦，跟吃药了一样，例假也不来，我这心都滴了嗓子眼儿了
3,好像得产后抑郁症了，想离婚，感觉各种烦
4,有了这个懒人烹饪锅我都爱上做饭啦！每天再也不用点外卖了傻瓜操做我这种厨房新手好适合


In [9]:
import csv
fw = open('results.csv', 'w',encoding='utf-8', newline='')
csv_writer = csv.writer(fw)
tts = '内容	乐的占比 	好的占比	怒的占比	哀的占比	惧的占比 	恶的占比	惊的占比	标签'
csv_writer.writerow([x for x in tts.split()])
from tqdm import tqdm
for filename in tqdm(df['内容'].values):
    info = getData(filename)
    csv_writer.writerow(list(info))
fw.close()

100%|████████████████████████████████████████████████████████████████████████████| 17539/17539 [19:40<00:00, 14.86it/s]


In [10]:
dfx = pd.read_csv('results.csv')

In [11]:
dfx.head()

,内容,乐的占比,好的占比,怒的占比,哀的占比,惧的占比,恶的占比,惊的占比,标签
0,打工人，再也不用点外卖了！有了它，想吃啥就吃啥提前注水，开机加热半小时就可以吃到麻麻做的热乎...,2,2,0,0,0,1,0,乐
1,宝宝辅食山药虾泥这个味道真的很好我家不爱吃辅食的宝宝都喜欢?-好吃-宝妈们可以尝试一下哦山药...,0,3,0,0,0,0,0,好
2,上火了，口苦，跟吃药了一样，例假也不来，我这心都滴了嗓子眼儿了,0,0,1,0,0,0,0,怒
3,好像得产后抑郁症了，想离婚，感觉各种烦,0,0,0,0,0,1,0,恶
4,有了这个懒人烹饪锅我都爱上做饭啦！每天再也不用点外卖了傻瓜操做我这种厨房新手好适合,0,1,0,0,0,0,0,好


In [12]:
label2info = {}
for l in dfx['标签']:
    label2info[l] = label2info.setdefault(l, 0) + 1

In [13]:
label2info

{'乐': 9151, '好': 5538, '怒': 69, '恶': 2534, '惧': 209, '惊': 38}

In [14]:
from pyecharts import options as opts
from pyecharts.charts import Page, Pie
l1 = []
num = []
for k, v in label2info.items():
    l1.append(k)
    num.append(v)
c = (
        Pie()
        .add("", [list(z) for z in zip(l1,num)])
        .set_global_opts(title_opts=opts.TitleOpts(title="细粒度情感占比"))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    )
c.render_notebook()

In [15]:
dfx.to_excel('final_results.xlsx', index=False)